# Module 1 - Foundational Models

This introductory module serves as a comprehensive primer on building functional AI agents using LangChain, moving systematically from basic initialization to complex multi-modal interactions. You will begin by learning to configure chat models and refine their behavior through system prompts and engineering techniques, before progressing to the critical step of integrating external tools to expand the agent's utility. The lab also covers the implementation of short-term memory for fluid conversations and the use of audio and visual inputs to create a more versatile assistant. Ultimately, the course culminates in a practical project where you will synthesize your skills to develop a web-connected culinary assistant capable of generating recipes from specific ingredients.

## 00. Getting started

Load the required libraries and environment variables:

In [ ]:
!cp /home/ubuntu/.env_vars .env
!pip install dotenv
!pip install langchain-openai

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 01. Initialising and invoking a model

The foundation of any agent is the chat model, described as the conductor of the orchestra or the system's "thinking brain". The interaction with models should be straightforward and interchangeable.
Use the `AzureChatOpenAI` function to select a model (e.g., gpt-4o).

In our lab, the model has been defined using the `AZURE_OPENAI_DEPLOYMENT` environment variable:

In [ ]:
!echo $AZURE_OPENAI_DEPLOYMENT

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

Then, you interact with the model by invoking it with a question. The response is returned as an AI message object, where the primary answer is stored within the content key.

In [ ]:
response = model.invoke("What's the capital of the Moon?")

response

In [ ]:
print(response.content)

In [ ]:
from pprint import pprint

pprint(response.response_metadata)

## 02. Customising your Model

Beyond the raw text response, models provide metadata (like token usage) and can be fine-tuned via specific parameters to control their behavior.  
• Temperature: Controls randomness; higher numbers increase creativity, while lower numbers make the model more deterministic.  
• Max Tokens: Limits the length of the output.  
• Timeout & Retries: Timeout sets the maximum wait time for a response, and max_retries defines how many times the system should attempt a failed request.  

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=1.0
)

response = model.invoke("What's the capital of the Moon?")
print(response.content)

## 03. Model Providers

A key strength of LangChain is that it is model agnostic, meaning you can switch between different providers (OpenAI, Claude, Gemini) with minimal code changes.
You can swap an entire agent's "brain" by changing the model name in the initialization function or using specific provider libraries for newer models. Please refer to the following documentation to get details on supported providers: https://docs.langchain.com/oss/python/integrations/chat

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment="gpt-5.1",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

response = model.invoke("What's the capital of the Moon?")
print(response.content)

## 04. Initialising and invoking an agent

To move from a simple chat model to an agent, LangChain uses an abstraction built on top of LangGraph. The agent is initialized by passing the model as the primary argument.

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

agent = create_agent(model=model)

Then, when invoking the agent, you pass a dictionary containing a human message function `Human Communication` to specify the user's input.

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What's the capital of the Moon?")]}
)

Unlike a simple model call, the agent returns a dictionary of messages representing the full conversation history; the final answer is always the last message in that list.

In [ ]:
from pprint import pprint

pprint(response)

In the next command, response['messages'][-1] selects the final message in the agent’s message list (the latest turn), and .content accesses that message’s text; print(...) outputs that text to the notebook’s stdout.

In [ ]:
print(response['messages'][-1].content)

Because the system accepts a dictionary, you can pass a list of previous Human and AI messages to provide context or "gaslight" the model with prior (even if false) information.

In [ ]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What's the capital of the Moon?"),
    AIMessage(content="The capital of the Moon is Luna City."),
    HumanMessage(content="Interesting, tell me more about Luna City")]}
)

pprint(response)

## 05. Streaming Output

As agent systems grow complex, they must handle multiple messages and the "perceived latency" of slow response times. Agent response times can jump from milliseconds to seconds or minutes. To improve the user experience, use streaming tokens (printing text as it is generated) rather than waiting for the full answer

In [ ]:
for token, metadata in agent.stream(
    {"messages": [HumanMessage(content="Tell me all about Luna City, the capital of the Moon")]},
    stream_mode="messages"
):

    # token is a message chunk with token content
    # metadata contains which node produced the token
    
    if token.content:  # Check if there's actual content
        print(token.content, end="", flush=True)  # Print token